In [53]:
import plotly.graph_objs as go
import pandas as pd
from plotly.offline import iplot
import numpy as np
import statistics 
import plotly.express as px
from numpy.lib import recfunctions as rfn

In [85]:
e7_t01_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T01_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t02_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T02_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t07_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T07_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t08_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T08_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t15_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T15_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t16_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T16_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame

cell_types = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_E7_cell_type.txt", sep="\t") #pandas DataFrame for cell types

In [86]:
cell_types_list = ['Alpha','Beta','Delta','Gamma','Epsilon'] #the cell types of interest for this work
cell_types = cell_types[cell_types['CellType'].isin(cell_types_list)] # parsing sampele type list, to only contain the cell types of interest
cell_types.reset_index(drop=True, inplace=True)

updated_list = []
for t in cell_types['SampleID'].values:
    if t in ['T01', 'T02']:
        updated_list.append('0Hr')
    elif t in ['T07','T08']:
        updated_list.append('2Hr')
    elif t in ['T15','T16']:
        updated_list.append('12Hr')
    else:
        updated_list.append('NA')

        
cell_types['TimePoint'] = updated_list

In [87]:
cell_types

,CellID,SampleID,CellType,TimePoint
0,AAACCTGGTAGCCTAT-1,T01,Alpha,0Hr
1,AAACCTGTCAAACGGG-1,T01,Alpha,0Hr
2,AAACGGGCAGTATGCT-1,T01,Alpha,0Hr
3,AAAGATGAGGATGGTC-1,T01,Alpha,0Hr
4,AAAGATGGTTAAGGGC-1,T01,Alpha,0Hr
...,...,...,...,...
24700,TTCTACAGTGAGGCTA-1,T22,Gamma,NA
24701,TTCTTAGGTTGAGTTC-1,T22,Gamma,NA
24702,TTGGAACCACATAACC-1,T22,Gamma,NA
24703,TTTCCTCCAATCGGTT-1,T22,Gamma,NA


In [88]:
# Data dict
sample_dict = {
    'T01': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T01_logNorm.csv.gz", sep=",", index_col=0,low_memory=False), #pandas DataFrame,
    'T02': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T02_logNorm.csv.gz", sep=",", index_col=0,low_memory=False), #pandas DataFrame
    'T07': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T07_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T08': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T08_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T15': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T15_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T16': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T16_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
}

In [102]:
cells = pd.DataFrame([], columns = ['gene', 'mean_expr_1', 'mean_expr_2', 'cell_type', 'time_point'])

for cell_type in np.unique(cell_types['CellType']):
    if cell_type == "unknown":
        continue
    sub_cells = cell_types[cell_types.CellType == cell_type]
    for time_point in np.unique(cell_types['TimePoint']):
        ## To do -- deal with High / low glucose treatment
        sampls = np.unique(cell_types[cell_types.TimePoint == time_point]['SampleID'])
#         Get sample IDs
        counts_1 = sample_dict.get(sampls[0])
        counts_2 = sample_dict.get(sampls[1])
        if counts_1 is None or counts_2 is None:
            continue
        
        barcodes_1 = sub_cells[sub_cells.SampleID == sampls[0]]['CellID'].values
        barcodes_2 = sub_cells[sub_cells.SampleID == sampls[1]]['CellID'].values
        means_1 = np.mean(counts_1.loc[:, barcodes_1],axis=1) 
        means_2 = np.mean(counts_2.loc[:, barcodes_2],axis=1)
        joint_index = np.intersect1d(means_1.index.values, means_2.index.values)

        cells = cells.append( #look at the gene name for R1 and R2, output that and compare to see if they match or not*
            pd.DataFrame({
                'gene': joint_index,
                'mean_expr_1': means_1.loc[joint_index].values,
                'mean_expr_2': means_2.loc[joint_index].values,
                'cell_type': [cell_type] * len(joint_index),
                'time_point': [time_point] * len(joint_index)  
            }))

In [115]:
# cells.sort_values(by=['time_point'],inplace=True)
# cells
pd.Categorical(cells['time_point'], categories=('0Hr','2Hr','12Hr'), ordered=True)


list_ordering = ['0Hr','2Hr','12Hr']  
cells["time_point"] = pd.Categorical(cells["time_point"], categories=list_ordering) 



In [116]:
# cells[cells['time_point'].notnull() & (cells['time_point'] == "12Hr")]
cells

,gene,mean_expr_1,mean_expr_2,cell_type,time_point
0,A1BG,0.602543,0.653069,Alpha,0Hr
5,AACS,0.107896,0.131969,Gamma,0Hr
4,AAAS,0.066496,0.041468,Gamma,0Hr
3,A4GALT,0.000000,0.006724,Gamma,0Hr
2,A2M,0.000000,0.000000,Gamma,0Hr
...,...,...,...,...,...
330,ANPEP,NaN,NaN,Epsilon,2Hr
331,ANTKMT,NaN,NaN,Epsilon,2Hr
332,ANXA1,NaN,NaN,Epsilon,2Hr
322,ANKRD9,NaN,NaN,Epsilon,2Hr


In [120]:
import plotnine as plt9
from plotnine import *
from plotnine.data import *

%matplotlib inline

plt9.options.figure_size = (22,10)

p=plt9.ggplot(cells, plt9.aes(
    x='mean_expr_1', 
    y = 'mean_expr_2')) + plt9.geom_point(
        alpha=0.50) + geom_abline(intercept=0, 
    slope=1) + plt9.facet_grid('cell_type ~ time_point') + labs(title="Counts comparison between technical replicates (E07-Low Glucose)",
                                                                x="Replicate-1",y="Replicate-2",size="Gene",color="Gene") + theme_classic(base_family = "Helvetica")+geom_point(color='darkgreen')


p.save(filename = 'E07_low_glucose.png', height=5, width=5, units = 'in', dpi=1000)

/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 5 x 5 in image.
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: E07_low_glucose.png
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/layer.py:467: PlotnineWarning: geom_point : Removed 27703 rows containing missing values.
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/layer.py:467: PlotnineWarning: geom_point : Removed 27703 rows containing missing values.
